In [ ]:
import os
import scipy
import numpy as np
import trimesh
from trimesh import viewer
from trimesh.transformations import translation_matrix, rotation_matrix, concatenate_matrices
from scipy.spatial.transform import Rotation

np.set_printoptions(precision=3, suppress=True, linewidth=np.inf)


In [ ]:
mesh1 = trimesh.exchange.load.load('wolf1.ply')
mesh2 = mesh1.copy()

# apply an arbitrary transformation to mesh1
T = translation_matrix([-11, 0, 11])
xaxis, yaxis, zaxis = [1, 0, 0], [0, 1, 0], [0, 0, 1]
Rx = rotation_matrix(0, xaxis)
Ry = rotation_matrix(0, yaxis)
Rz = rotation_matrix(0, zaxis)
M = concatenate_matrices(T, Rx, Ry, Rz)
mesh1.apply_transform(M)

# write scene to html
if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html((trimesh.util.concatenate([mesh1, mesh2])).scene())
with open("scene.html", "w") as file:
    file.write(html)


In [ ]:
def register(source, target, max_iter=100):
    # resulting transformation is (R, t)
    transform = (np.eye(3), np.zeros(3))
    x = source.vertices
    y = target.vertices
    N = len(x)
    for iter in range(max_iter):
        # applies transformation to original x
        x_t = x.dot(transform[0]) + transform[1]
        # build a matrix equivalent to cross product
        X_t = np.stack([
            np.stack((np.zeros(len(x_t)), -x_t[:, 2], x_t[:, 1]), axis=1),
            np.stack((x_t[:, 2], np.zeros(len(x_t)), -x_t[:, 0]), axis=1),
            np.stack((-x_t[:, 1], x_t[:, 0], np.zeros(len(x_t))), axis=1)
        ], axis=1)
        # build blocks of sparse coefficient matrix A
        A_rr = np.einsum('abc,adc->bd', X_t, X_t)
        A_tr = -np.sum(X_t, axis=0)
        A_rt = A_tr.T
        A_tt = N * np.eye(3)
        A_rr = scipy.sparse.csr_matrix(A_rr)
        A_tr = scipy.sparse.csr_matrix(A_tr)
        A_rt = scipy.sparse.csr_matrix(A_rt)
        A_tt = scipy.sparse.csr_matrix(A_tt)
        # assemble coefficient matrix A
        A = scipy.sparse.bmat([[A_rr, A_tr],
                               [A_rt, A_tt]], format='csr')
        # build blocks of result vector b
        b_r = -np.einsum('abc,ac->b', X_t, y)
        b_t = np.sum(y - x_t, axis=0)
        # assemble result vector b
        b = np.concatenate([b_r, b_t])
        # solve system of linear equations Ax = b using conjugate gradient method
        solution, info = scipy.sparse.linalg.cg(A, b, tol=1e-10, maxiter=1000)
        # ensure convergence
        assert info == 0
        # update resulting transformation
        r = solution[:3]
        t = solution[3:]
        R = Rotation.from_euler('xyz', r).as_matrix()
        # ensure that R is a rotation matrix
        assert np.isclose(np.linalg.det(R), 1.0)
        transform = (np.dot(transform[0], R), transform[1] + t)
        print(iter, '\n', transform, '\n')
    return transform


In [ ]:
R, t = register(mesh1, mesh2, max_iter=10)
print('ground truth\n', M[:3, :3], M[:3, 3])
trans_mesh = trimesh.base.Trimesh(mesh1.vertices.dot(R) + t, mesh1.faces)

if os.path.exists("scene.html"):
    os.remove("scene.html")
html = viewer.notebook.scene_to_html(trimesh.util.concatenate([trans_mesh, mesh2]).scene())
with open("scene.html", "w") as file:
    file.write(html)
